In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,2983.21,0.07,-0.00,-0.12,0.05,0.14,-0.03,-0.04,0.02,-0.03,...,-0.00,-0.01,-0.03,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00
2,0.07,4010.05,-0.03,-0.25,-0.07,-0.16,0.02,0.02,-0.00,-0.11,...,-0.00,0.00,-0.02,-0.02,0.00,-0.02,0.00,-0.00,-0.00,0.00
3,-0.00,-0.03,1140.68,-0.02,-0.00,0.05,0.03,0.02,0.01,0.06,...,-0.00,0.01,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00
4,-0.12,-0.25,-0.02,4662.57,0.10,0.07,-0.05,0.04,-0.02,-0.02,...,0.01,0.00,0.01,0.01,0.00,-0.01,0.00,-0.00,0.00,-0.00
5,0.05,-0.07,-0.00,0.10,2871.21,-0.13,-0.02,0.01,-0.02,-0.06,...,0.00,0.01,0.00,-0.00,-0.00,-0.01,0.00,-0.00,0.01,0.00
6,0.14,-0.16,0.05,0.07,-0.13,2048.79,0.00,-0.01,0.03,0.01,...,-0.00,0.01,0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00
7,-0.03,0.02,0.03,-0.05,-0.02,0.00,1140.07,0.00,0.01,0.00,...,-0.01,0.01,-0.01,-0.01,0.00,-0.00,0.00,0.00,0.00,-0.00
8,-0.04,0.02,0.02,0.04,0.01,-0.01,0.00,87.25,0.00,-0.01,...,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00
9,0.02,-0.00,0.01,-0.02,-0.02,0.03,0.01,0.00,407.71,0.00,...,-0.00,0.00,0.01,-0.01,-0.00,-0.01,-0.00,0.00,-0.00,-0.00
10,-0.03,-0.11,0.06,-0.02,-0.06,0.01,0.00,-0.01,0.00,1568.63,...,0.00,-0.00,0.01,-0.00,0.00,0.01,-0.00,0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,0.00002,-0.00000,-0.00003,0.00002,0.00006,-0.00002,-0.00007,0.00001,-0.00001,...,-0.00000,-0.00001,-0.00002,-0.00000,0.00000,-0.00000,0.00000,0.00000,0.00001,0.00001
2,0.00002,1.00000,-0.00002,-0.00006,-0.00002,-0.00006,0.00001,0.00003,-0.00000,-0.00004,...,-0.00000,0.00000,-0.00001,-0.00002,0.00000,-0.00001,0.00000,-0.00000,-0.00001,0.00000
3,-0.00000,-0.00002,1.00000,-0.00001,-0.00000,0.00003,0.00002,0.00006,0.00002,0.00004,...,-0.00001,0.00002,0.00000,0.00001,-0.00001,0.00000,0.00001,0.00001,-0.00000,0.00000
4,-0.00003,-0.00006,-0.00001,1.00000,0.00003,0.00002,-0.00002,0.00006,-0.00001,-0.00001,...,0.00003,0.00000,0.00001,0.00001,0.00000,-0.00001,0.00000,-0.00001,0.00001,-0.00001
5,0.00002,-0.00002,-0.00000,0.00003,1.00000,-0.00005,-0.00001,0.00002,-0.00002,-0.00003,...,0.00001,0.00001,0.00000,-0.00000,-0.00000,-0.00001,0.00000,-0.00000,0.00001,0.00000
6,0.00006,-0.00006,0.00003,0.00002,-0.00005,1.00000,0.00000,-0.00001,0.00003,0.00001,...,-0.00000,0.00001,0.00000,-0.00000,0.00002,-0.00000,0.00000,-0.00000,-0.00000,0.00001
7,-0.00002,0.00001,0.00002,-0.00002,-0.00001,0.00000,1.00000,0.00001,0.00001,0.00000,...,-0.00003,0.00001,-0.00001,-0.00001,0.00000,-0.00000,0.00002,0.00000,0.00000,-0.00000
8,-0.00007,0.00003,0.00006,0.00006,0.00002,-0.00001,0.00001,1.00000,0.00002,-0.00001,...,-0.00001,0.00002,-0.00001,-0.00002,-0.00001,-0.00001,0.00000,0.00001,0.00000,0.00000
9,0.00001,-0.00000,0.00002,-0.00001,-0.00002,0.00003,0.00001,0.00002,1.00000,0.00000,...,-0.00000,0.00002,0.00001,-0.00002,-0.00001,-0.00002,-0.00002,0.00000,-0.00001,-0.00000
10,-0.00001,-0.00004,0.00004,-0.00001,-0.00003,0.00001,0.00000,-0.00001,0.00000,1.00000,...,0.00000,-0.00000,0.00001,-0.00000,0.00001,0.00001,-0.00000,0.00000,-0.00000,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.4532918781569906

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.89034655e+05 2.87417685e+05 1.59714791e+05 1.47061974e+05
 9.79555055e+04 8.10165478e+04 7.52190733e+04 6.96818643e+04
 5.41268481e+04 5.27786021e+04 4.85856458e+04 4.71232134e+04
 4.38221151e+04 4.02045058e+04 3.76018908e+04 3.01650817e+04
 2.87831320e+04 2.53525087e+04 2.47591037e+04 2.36434102e+04
 2.28296201e+04 2.14454678e+04 2.01819596e+04 1.85282284e+04
 1.46817745e+04 1.38402987e+04 1.16743050e+04 1.03641592e+04
 8.87278742e+03 8.32909180e+03 7.43547262e+03 6.98848534e+03
 6.63405438e+03 6.08462410e+03 5.66229978e+03 5.18201115e+03
 4.84613402e+03 4.57887744e+03 4.00695059e+03 3.33321325e+03
 3.02807008e+03 2.84714153e+03 2.51225603e+03 2.12634122e+03
 1.74369857e+03 1.42942477e+03 1.28743560e+03 1.16568584e+03
 6.77203929e+02 2.58937766e+02 1.95799896e+02 1.10826264e+02
 6.00947543e+01 2.78573281e+01 5.15936034e+00 5.54380503e-26]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-0.017587,-0.055744,-0.004170,0.997719,-0.015952,-0.008707,-0.004176,-0.000291,-0.001413,-0.006081,...,-0.000199,-0.000874,-0.001709,-0.001115,-0.000418,-0.001239,-0.000066,-0.000192,-0.000397,-0.000049
1,-0.029181,0.997019,-0.005338,0.053600,-0.025762,-0.012037,-0.005314,-0.000369,-0.001766,-0.008006,...,-0.000253,-0.001094,-0.002155,-0.001406,-0.000522,-0.001553,-0.000083,-0.000239,-0.000499,-0.000061
2,0.978086,0.020679,-0.005757,0.013672,-0.199978,-0.016693,-0.005767,-0.000388,-0.001779,-0.009382,...,-0.000255,-0.001106,-0.002195,-0.001414,-0.000524,-0.001564,-0.000083,-0.000240,-0.000499,-0.000061
3,0.194231,0.026604,-0.008894,0.018032,0.976600,-0.027997,-0.008900,-0.000570,-0.002732,-0.014806,...,-0.000385,-0.001675,-0.003333,-0.002153,-0.000799,-0.002387,-0.000126,-0.000365,-0.000758,-0.000093
4,0.028064,0.012261,-0.009218,0.009455,0.033200,-0.056775,-0.009146,-0.000532,-0.002569,-0.017621,...,-0.000361,-0.001576,-0.003162,-0.002023,-0.000747,-0.002251,-0.000118,-0.000339,-0.000713,-0.000087
5,0.016688,0.008473,-0.008989,0.006702,0.018726,-0.196829,-0.008907,-0.000479,-0.002374,-0.019443,...,-0.000328,-0.001437,-0.002900,-0.001857,-0.000683,-0.002053,-0.000107,-0.000310,-0.000646,-0.000079
6,0.017444,0.009486,-0.011234,0.007409,0.019722,0.950576,-0.011239,-0.000595,-0.002873,-0.026218,...,-0.000401,-0.001748,-0.003553,-0.002262,-0.000824,-0.002510,-0.000131,-0.000379,-0.000790,-0.000096
7,0.021477,0.012009,-0.016526,0.009534,0.024015,0.205990,-0.016534,-0.000830,-0.004055,-0.041987,...,-0.000564,-0.002475,-0.005056,-0.003195,-0.001168,-0.003554,-0.000184,-0.000535,-0.001115,-0.000136
8,0.002685,0.001618,-0.003688,0.001316,0.002961,0.009292,-0.003610,-0.000155,-0.000782,-0.016982,...,-0.000111,-0.000476,-0.000968,-0.000618,-0.000222,-0.000680,-0.000035,-0.000099,-0.000210,-0.000026
9,0.013402,0.008151,-0.019201,0.006616,0.014462,0.042479,-0.019140,-0.000808,-0.003996,-0.095045,...,-0.000548,-0.002402,-0.004989,-0.003118,-0.001139,-0.003462,-0.000178,-0.000517,-0.001079,-0.000131


In [19]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = np.sort(np.concatenate(cos_dist))[:len(GROUPS)]
cos_dist


array([0.00228053, 0.00298081, 0.0087733 , 0.02191403, 0.02339988,
       0.0281637 , 0.04272165, 0.04942426, 0.07030461, 0.07291617,
       0.0856101 , 0.10115506, 0.10146723, 0.10623397, 0.10659473,
       0.11274046, 0.1161509 , 0.14057283, 0.14835912, 0.16939868,
       0.18575781, 0.1967032 , 0.1985666 , 0.19907851, 0.21082378,
       0.21445887, 0.21672508, 0.22542708, 0.22553529, 0.22831536,
       0.23475234, 0.2442011 , 0.24443939, 0.24445309, 0.25917949,
       0.26417951, 0.2683017 , 0.26938352, 0.27198851, 0.27459783,
       0.27827865, 0.27857788, 0.27974449, 0.28610167, 0.28748218,
       0.30442426, 0.30522904, 0.32419159, 0.33534571, 0.35992356,
       0.36396822, 0.37479814, 0.41252687, 0.44296306, 0.45603363,
       0.45727766])